The Wallaroo 101 tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-101).

## Introduction

Welcome to the Wallaroo, the fastest, easiest, and most efficient production ready machine learning system.

This tutorial is created to help you get started with Wallaroo right away.  We'll start with a brief explanation of how Wallaroo works, then provide the credit card fraud detection model so you can see it working.

This guide assumes that you've installed Wallaroo in your cloud Kubernetes cluster.  This can be either:

* Amazon Web Services (AWS)
* Microsoft Azure
* Google Cloud Platform

For instructions on setting up your cloud Kubernetes environment, check out the [Wallaroo Environment Setup Guides](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-setup-environment/) for your particular cloud provider.

### How to Use This Notebook

It is recommended that you run this notebook command at a time so you can see the results and make any changes you need based on your own environment.

## SDK Introduction

The Wallaroo SDK lets you quickly get your models working with your data and getting results.  The typical flow follows these steps:

* **Connect**:  Connect to your Wallaroo Instance.
* **Create or Connect to a Workspace**:  Create a new workspace that will contain your models and pipelines, or connect to an existing one.
* **Upload or Use Existing Models**:  Upload your models to your workspace, or use ones that have already been uploaded.
* **Create or Use Existing Pipelines**:  Create or use an existing pipeline.  This is where you'll set the **steps** that will ingest your data, submit it through each successive model, then return a result.
* **Deploy Your Pipeline**:  Deploying a pipeline allocates resources from your Kubernetes environment for your models.
* **Run an Inference**:  This is where it all comes together.  Submit data through your pipeline either as a file or to your pipeline's deployment url, and get results.
* **Undeploy Your Pipeline**:  This returns the Kubernetes resources your pipeline used back to the Kubernetes environment.

For a more detailed rundown of the Wallaroo SDK, see the [Wallaroo SDK Essentials Guide](https://docs.wallaroo.ai/wallaroo-sdk/wallaroo-sdk-essentials-guide/).

### Introduction to Workspaces

A Wallaroo **Workspace** allows you to manage a set of models and pipelines.  You can assign users to a workspace as either an **owner** or **collaborator**.

When working within the Wallaroo SDK, the first thing you'll do after connecting is either create a workspace or set an existing workspace your **current workspace**.  From that point on, all models uploaded and pipelines created or used will be in the context of the current workspace.

### Introduction to Models

A Wallaroo **model** is a trained Machine Learning model that is uploaded to your current workspace.  These are the engines that take in data, run it through whatever process they have been trained for, and return a result.

Models don't work in a vacuum - they are allocated to a pipeline as detailed in the next step.

### Introduction to Pipelines

A Wallaroo **pipeline** is where the real work occurs.  A pipeline contains a series of **steps** - sequential sets of models which take in the data from the preceding step, process it through the model, then return a result.  Some models can be simple, such as the `cc_fraud` example listed below where the pipeline has only one step:

* Step 0: Take in data
* Step 1: Submit data to the model `ccfraudModel`.
* Step Final:  Return a result

Some models can be more complex with a whole series of models - and those results can be submitted to still other pipeline.  You can make pipelines as simple or complex as long as it meets your needs.

Once a step is created you can add additional steps, remove a step, or swap one out until everything is running perfectly.

**Note**: The Community Edition of Wallaroo limits users to two active pipelines, with a maximum of five steps per pipeline.

With all of that introduction out of the way, let's proceed to our Credit Card Detection Model.

This example will demonstrate how to use Wallaroo to detect credit card fraud through a trained model and sample data.  By the end of this example, you'll be able to:

* Start the Wallaroo client.
* Create a workspace.
* Upload the credit card fraud detection model to the workspace.
* Create a new pipeline and set it to our credit card fraud detection model.
* Run a smoke test to verify the pipeline and model is working properly.
* Perform a bulk inference and display the results.
* Undeploy the pipeline to get back the resources from our Kubernetes cluster.

This example and sample data comes from the Machine Learning Group's demonstration on [Credit Card Fraud detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [19]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd
import json
from IPython.display import display

# used to display dataframe information without truncating
pd.set_option('display.max_colwidth', None)

In [2]:
wallaroo.__version__

'0.1.dev1+gd948e20.d20230118'

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

# # SSO login through keycloak

# wallarooPrefix = "YOUR PREFIX"
# wallarooSuffix = "YOUR SUFFIX"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

Please log into the following URL in a web browser:

	https://sparkly-apple-3026.keycloak.wallaroo.community/auth/realms/master/device?user_code=THJX-ZQMG

Login successful!


In [4]:
import os
os.environ["ARROW_ENABLED"]="True"
if "ARROW_ENABLED" in os.environ:
    arrowEnabled = os.environ["ARROW_ENABLED"]
else:
    arrowEnabled = False

In [5]:
print(arrowEnabled)

True


## Create a New Workspace

Next we're going to create a new workspace called `ccfraudworkspace` for our model, then set it as our current workspace context.  We'll be doing this through the SDK, but here's an example of doing it through the Wallaroo dashboard.

The method we'll introduce below will either **create** a new workspace if it doesn't exist, or **select** an existing workspace.  So if you create the workspace `ccfraudworkspace` then you're covered either way.

The first part is to return to your Wallaroo Dashboard.  In the top navigation panel next to your user name there's a drop down with your workspaces.  In this example it just has "My Workspace".  Select **View Workspaces**.

![Select View Workspaces](./images/wallaroo-101/wallaroo-dashboard-select-view-workspaces.png)

From here, enter the name of our new workspace as `ccfraud-workspace`.  If it already exists, you can skip this step.

* **IMPORTANT NOTE**:  Workspaces do not have forced unique names.  It is highly recommended to use an existing workspace when possible, or establish a naming convention for your workspaces to keep their names unique to remove confusion with teams.

![Create ccfraud-workspace](./images/wallaroo-101/wallaroo-dashboard-create-workspace-ccfraud.png)

Once complete, you'll be able to select the workspace from the drop down list in your dashboard.

![ccfraud-workspace exists](./images/wallaroo-101/wallaroo-dashboard-ccfraud-workspace-exists.png)

Just for the sake of this tutorial, we'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

When we create our new workspace, we'll save it in the Python variable `workspace` so we can refer to it as needed.

In [6]:
prefix="jch-"
workspace_name = f'{prefix}ccfraudworkspacearrow'
pipeline_name = f'{prefix}ccfraudpipelinearrow'
model_name = f'{prefix}ccfraudmodelarrow'
model_file_name = './ccfraud.onnx'

In [7]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [8]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'jch-ccfraudworkspacearrow', 'id': 23, 'archived': False, 'created_by': '39bf22e5-bb55-40ce-b1ab-36eed7a29150', 'created_at': '2023-02-09T21:49:18.730848+00:00', 'models': [], 'pipelines': []}

In [9]:
wl.list_workspaces()

Name,Created At,Users,Models,Pipelines
john.hansarick@wallaroo.ai - Default Workspace,2023-02-09 15:19:32,['john.hansarick@wallaroo.ai'],0,0
jch-abtestworkspace,2023-02-09 15:56:46,['john.hansarick@wallaroo.ai'],2,1
jch-ccfraudworkspacearrow,2023-02-09 21:49:18,['john.hansarick@wallaroo.ai'],0,0


Just to make sure, let's list our current workspace.  If everything is going right, it will show us we're in the `ccfraud-workspace`.

In [10]:
wl.set_current_workspace(workspace)
wl.get_current_workspace()

{'name': 'jch-ccfraudworkspacearrow', 'id': 23, 'archived': False, 'created_by': '39bf22e5-bb55-40ce-b1ab-36eed7a29150', 'created_at': '2023-02-09T21:49:18.730848+00:00', 'models': [], 'pipelines': []}

## Upload a model

Our workspace is created.  Let's upload our credit card fraud model to it.  This is the file name `ccfraud.onnx`, and we'll upload it as `ccfraudmodel`.  The credit card fraud model is trained to detect credit card fraud based on a 0 to 1 model:  The closer to 0 the less likely the transactions indicate fraud, while the closer to 1 the more likely the transactions indicate fraud.


Since we're already in our default workspace `ccfraudworkspace`, it'll be uploaded right to there.  Once that's done uploading, we'll list out all of the models currently deployed so we can see it included.

In [11]:
ccfraud_model = wl.upload_model(model_name, model_file_name).configure()

We can verify that our model was uploaded by listing the models uploaded to our Wallaroo instance with the `list_models()` command.  Note that since we uploaded this model before, we now have different versions of it we can use for our testing.

In [12]:
wl.list_models()

Name,# of Versions,Owner ID,Last Updated,Created At
jch-ccfraudmodelarrow,1,"""""",2023-02-09 21:49:28.449044+00:00,2023-02-09 21:49:28.449044+00:00


## Create a Pipeline

With our model uploaded, time to create our pipeline and deploy it so it can accept data and process it through our `ccfraudmodel`.  We'll call our pipeline `ccfraudpipeline`.

* **NOTE**:  Pipeline names must be unique.  If two pipelines are assigned the same name, the new pipeline is created as a new **version** of the pipeline.

In [13]:
ccfraud_pipeline = wl.build_pipeline(pipeline_name)

Now our pipeline is set.  Let's add a single **step** to it - in this case, our `ccfraud_model` that we uploaded to our workspace.

In [14]:
ccfraud_pipeline.add_model_step(ccfraud_model)

name,jch-ccfraudpipelinearrow
created,2023-02-09 21:49:31.767814+00:00
last_updated,2023-02-09 21:49:31.767814+00:00
deployed,(none)
tags,
versions,c69f1379-6b83-4cd9-9fed-0ff443990fe2
steps,


And now we can deploy our pipeline and assign resources to it.  This typically takes about 45 seconds once the command is issued.

In [15]:
ccfraud_pipeline.deploy()

Waiting for deployment - this will take up to 45s ................ ok


name,jch-ccfraudpipelinearrow
created,2023-02-09 21:49:31.767814+00:00
last_updated,2023-02-09 21:49:37.553573+00:00
deployed,True
tags,
versions,"115e5663-90ca-44a7-b2c6-ca5cc047cc88, c69f1379-6b83-4cd9-9fed-0ff443990fe2"
steps,jch-ccfraudmodelarrow


We can see our new pipeline with the `status()` command.

In [16]:
ccfraud_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.48.2.4',
   'name': 'engine-6c5f676489-gqq94',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'jch-ccfraudpipelinearrow',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'jch-ccfraudmodelarrow',
      'version': '782d6860-e92d-4ded-8d98-4dfd7c4549ba',
      'sha': 'bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.48.2.3',
   'name': 'engine-lb-74b4969486-gfjpk',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

## Running Interfences

With our pipeline deployed, let's run a smoke test to make sure it's working right.  We'll run an inference through our pipeline from the file `smoke_test.json` and see the results.  This should give us a result near 0 - not likely a fraudulent activity.

In [20]:
if arrowEnabled:
    result = ccfraud_pipeline.infer_from_file('./smoke_test.df.json')
else:
    result = ccfraud_pipeline.infer_from_file('./smoke_test.json')
display(result)

,time,out.dense_1,check_failures,metadata.last_model
0,1675979496603,[0.0014974177],[],"{""model_name"":""jch-ccfraudmodelarrow"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}"


Looks good!  Time to run the real test on some real data.  Run another inference this time from the file `high_fraud.json` and let's see the results.  This should give us an output that indicates a high level of fraud - well over 90%.

In [21]:
if arrowEnabled:
    result = ccfraud_pipeline.infer_from_file('./high_fraud.df.json')
else:
    result = ccfraud_pipeline.infer_from_file('./high_fraud.json')
display(result)

,time,out.dense_1,check_failures,metadata.last_model
0,1675979523693,[0.981199],[],"{""model_name"":""jch-ccfraudmodelarrow"",""model_sha"":""bc85ce596945f876256f41515c7501c399fd97ebcb9ab3dd41bf03f8937b4507""}"


Now that we've tested our pipeline, let's run it with something larger.  We have two batch files - `cc_data_1k.json` that contains 1,000 credit card records to test for fraud.  The other is `cc_data_10k.json` which has 10,000 credit card records to test.

First let's run a batch result for `cc_data_1k.json` and see the results.  Inferences are returned as the [InferenceResult object](https://docs.wallaroo.ai/wallaroo-sdk/wallaroo-sdk-essentials-guide/#inferenceresult-object).  We'll retrieve the InferenceResult object and store it into a variable.

In [63]:
outputs = None
if arrowEnabled:
    outputs = ccfraud_pipeline.infer_from_file('./cc_data_10k.df.json')
else:
    outputs = ccfraud_pipeline.infer_from_file('./cc_data_10k.json')

Now we can isolate just the output with the `.data()` method, then isolate it down to just the results likely to be fraud.

In [147]:
# some differences here:
# if arrow, it's a dataframe, so filter first then display the results
# if not arrow, use the lamba stuff

if arrowEnabled:
    filter = [elt[0] > 0.75 for elt in outputs['out.dense_1']]
    display(outputs.loc[filter, ['out.dense_1']])
    #df[['a1_count', 'a1_mean', 'a1_std']].values
    
else:
    sequence = output[0].data()
    result = filter(lambda x: x > 0.75, sequence[0])
    print(list(result))

,out.dense_1
0,[0.99300325]
1,[0.99300325]
2,[0.99300325]
3,[0.99300325]
161,[1.0]
941,[0.9873102]
1445,[1.0]
2092,[0.99999]
2220,[0.91080534]
4135,[0.98877275]


We can also retrieve the inputs from our InferenceResult object through the `input_data()` method as follows - in this case, just the first record.

In [154]:
if arrowEnabled:
    ccfraud_pipeline.logs(limit=50)
    
else:
    output[0].input_data()["tensor"][0]

## Batch Deployment through a Pipeline Deployment URL

This next step requires some manual use.  We're going to have `ccfraud_pipeline` display its deployment url - this allows us to submit data through a HTTP interface and get the results back.

First we'll request the url with the `_deployment._url()` method.

* **IMPORTANT NOTE**:  The `_deployment._url()` method will return an **internal** URL when using Python commands from within the Wallaroo instance - for example, the Wallaroo JupyterHub service.  When connecting via an external connection, `_deployment._url()` returns an **external** URL.  External URL connections requires [the authentication be included in the HTTP request](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/), and that [Model Endpoints Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-model-endpoints-guide/) external endpoints are enabled in the Wallaroo configuration options.

In [ ]:
deploy_url = ccfraud_pipeline._deployment._url()

The API connection details can be retrieved through the Wallaroo client `mlops()` command.  This will display the connection URL, bearer token, and other information.  The bearer token is available for one hour before it expires.

For this example, the API connection details will be retrieved, then used to submit an inference request through the external inference URL retrieved earlier.

In [ ]:
connection =wl.mlops().__dict__
token = connection['token']
token

Copy and paste the results above into the curl command, replacing the {YOUR URL HERE} with your deploy url for `ccfraud_pipeline`, and uncomment it.

In [ ]:
!curl -X POST {deploy_url} -H "Authorization: Bearer {token}" -H "Content-Type:application/json" --data @cc_data_10k.json > curl_response.txt

With our work in the pipeline done, we'll undeploy it to get back our resources from the Kubernetes cluster.  If we keep the same settings we can redeploy the pipeline with the same configuration in the future.

In [ ]:
ccfraud_pipeline.undeploy()

And there we have it!  Feel free to use this as a template for other models, inferences and pipelines that you want to deploy with Wallaroo!